**Note: This is my EDA.  It's not pretty or nice and probably has some mistakes along the way.  Check out the associated `.py` file for a linted/formatted version of the solution.**

In [7]:
from functools import reduce

import polars as pl

from utils import get_data

# The Investigator

(JAMES: Emphasis below mine when I think something will be useful.)

REF: https://hanukkah.bluebird.sh/5784/1/

Sarah brought a cashier over. She said, “Joe here says that one of our customers is a skilled private investigator.”

Joe nodded, “They showed me their business card, and that’s what it said. **Skilled Private Investigator**. And their **phone number was their last name spelled out**. I didn’t know what that meant, but apparently before there were smartphones, people had to remember phone numbers or write them down. If you wanted a phone number that was easy-to-remember, you could get a number that spelled something using the letters printed on the phone buttons: like 2 has “ABC”, and 3 “DEF”, etc. And I guess this person had done that, so if you dialed the numbers corresponding to the letters in their name, it would call their phone number!

“I thought that was pretty cool. But I don’t remember their name, or anything else about them for that matter. I couldn’t even tell you if they were male or female.”

Sarah said, “This person seems like they are skilled at investigation. I need them to find Noah’s rug before the Hanukkah dinner. I don’t know how to contact them, but apparently they shop here at Noah’s Market.”

In [8]:
data = get_data()
df_customers = data["customers"]

In [9]:
LETTER_TO_BUTTON_LIST = [
    ("ABC", "2"),
    ("DEF", "3"),
    ("GHI", "4"),
    ("JKL", "5"),
    ("MNO", "6"),
    ("PQRS", "7"),
    ("TUV", "8"),
    ("WXYZ", "9")  
]
LETTER_TO_BUTTON_MAP = reduce(
    lambda x, y: x | y, 
    map(lambda x: {y: x[1] for y in x[0]}, LETTER_TO_BUTTON_LIST)
)

def last_name_to_phone_number(lastname: str) -> str:
    """Translate a name to a number of the form 'XXX-XXX-XXXX' via phone buttons."""
    if len(lastname) != 10:
        return ""
    lbm = "".join(LETTER_TO_BUTTON_MAP[letter] for letter in lastname.upper())
    return f"{lbm[:3]}-{lbm[3:6]}-{lbm[6:]}"

In [10]:
(df_customers
    .select(
        [
            pl.col("phone"),
            pl.col("name").str.split(by=" ").list.get(1).alias("last_name"),
        ])
    # Filter out all rows where last name != 10.
    .filter(pl.col("last_name").str.len_chars() == 10)
    # Create the last name number mapping.
    .select(
        [
            pl.col("phone"),
            pl.col("last_name")
            .map_elements(lambda x: last_name_to_phone_number(lastname=x))
            .alias("last_name_number"),
        ]
        # Filter out names not equal to phone number.
    )
    .filter(pl.col("last_name_number") == pl.col("phone"))
    .get_column("phone")
    .item()
)

'826-636-2286'